In [1]:
import pandas as pd
from tqdm import tqdm
import nltk.data
import re
import spacy
from tqdm import tqdm
import collections

nlp = spacy.load('en')

In [2]:
# df1 = pd.read_json('Data/New Age/newage_2016_10_01_to_2017_10_01.json')
# df2 = pd.read_json('Data/New Age/newage_2017_10_02_to_2018_04_20.json')

In [3]:
# df = pd.concat([df1, df2], ignore_index=True)

In [4]:
# df.head(1)

In [5]:
# df['news_content'][0]

In [6]:
# doc = nlp(df['news_content'][0])

In [7]:
# location = []
# organization = []
# person = []
# for entity in doc.ents:
#     if (entity.label_ == "LOC" or entity.label_ == "GPE"):
#         location.append(entity.text)
#     elif (entity.label_ == "ORG"):
#         organization.append(entity.text)
#     elif (entity.label_ == "PERSON"):
#         person.append(entity.text)
# print(list(set(location)), list(set(organization)), list(set(person)))

In [8]:
# def find_entities(text):
#     doc = nlp(text)
#     location = []
#     organization = []
#     person = []
#     for entity in doc.ents:
#         if (entity.label_ == "LOC" or entity.label_ == "GPE"):
#             location.append(entity.text)
#         elif (entity.label_ == "ORG"):
#             organization.append(entity.text)
#         elif (entity.label_ == "PERSON"):
#             person.append(entity.text)
#     location = list(set(location))
#     organization = list(set(organization))
#     person = list(set(person))
#     return location, organization, person

In [9]:
# tqdm.pandas()
# df['location_entities'], df['organization_entities'], df['person_entities'] = zip(*df['news_content'].progress_apply(find_entities))

In [10]:
# df.to_pickle('Data/New Age/NewAge_ent.pkl')

In [2]:
df = pd.read_pickle('Data/New Age/NewAge_ent.pkl')

In [12]:
df.head()

,category,date_published,image,news_content,news_id,newspaper,reporter,tags,title,url,location_entities,organization_entities,person_entities
0,Bangladesh,2016-10-01,"{'urls': [], 'captions': []}",Coordinator of 14-party alliance and health mi...,118,New Age,Bangladesh Sangbad Sangstha,"[General elections, Mohammad Nasim, national p...",No national polls before 2019: Nasim,http://www.newagebd.net/article/118/no-nationa...,[Naogaon],"[BSD, BNP, Workers Party, Advocate Enamul Haqu...","[Khairuzzaman Litan, Md Abdul Malek, Samyabadi..."
1,Entertainment,2016-10-01,{'urls': ['http://www.newagebd.com/files/recor...,"\n\t\t\t\t\t\t\tIn this July 4, 2015 file phot...",83,New Age,Associated Press,"[David Bowie , Rolling Stones]",Concert promoter in $200 million fraud is bip...,http://www.newagebd.net/article/83/concert-pro...,[Indianapolis],"[Worldwide Entertainment Inc., the Rolling Sto...","[Jack Utsick, Charlie Watts, Ronnie Wood, Keit..."
2,Entertainment,2016-10-01,{'urls': ['http://www.newagebd.com/files/recor...,"\n\t\t\t\t\t\t\tIn this March 7, 2000 photo, R...",86,New Age,Associated Press,"[Oscar Mayer Wiener , Richard Trentlage]",Man who got generations singing Oscar Mayer W...,http://www.newagebd.net/article/86/man-who-got...,"[Fox River Grove, Illinois, Libertyville, Ill....","[Modern Plastic Brooms, Oscar Mayer Wiener, Mc...","[Buckle, Richard Trentlage, Linda Bruun]"
3,Entertainment,2016-10-01,{'urls': ['http://www.newagebd.com/files/recor...,\n\t\t\t\t\t\t\tA 2015 file photo shows The Gr...,89,New Age,Associated Press,"[Henri Matisse, National Gallery of Art , Pabl...",National Gallery of Art's East Building set t...,http://www.newagebd.net/article/89/national-ga...,"[The Greek Slave, Washington, DC, The East Bui...","[the National Gallery of Art, AP]","[Virginia Dwan, Amedeo Modigliani, Georgia O'K..."
4,Entertainment,2016-10-01,{'urls': ['http://www.newagebd.com/files/recor...,\n\t\t\t\t\t\t\tThis file photo taken on April...,90,New Age,Agence France-Presse,"[Bob Dylan, European Commission, Michael Jacks...",Sony closes on Michael Jackson's stake in mus...,http://www.newagebd.net/article/90/sony-closes...,"[Jackson, US, Kanye West]","[Music Publishing, The European Commission, So...","[Marvin Gaye, Lady Gaga, Bob Dylan, Michael Ja..."


In [15]:
def remove_whitespace(entities):
    entities = [item.strip() for item in entities if not bool(re.match(r"^\s+$", item))]
    return entities

In [16]:
df['location_entities'] = df['location_entities'].apply(remove_whitespace)
df['organization_entities'] = df['organization_entities'].apply(remove_whitespace)
df['person_entities'] = df['person_entities'].apply(remove_whitespace)

In [17]:
def name_disambiguation(names):
    disambiguated_names = []
    for name in names:
        if (name == "Khaleda" or name == "Zia" or name == "Begum Khaleda Zia" or name == "Khaleda Zia’s"):
            disambiguated_names.append("Khaleda Zia")
        elif (name == "Hasina" or name == "Sheikh"):
            disambiguated_names.append("Sheikh Hasina")
        elif (name == "Fakhrul"):
            disambiguated_names.append("Mirza Fakhrul Islam Alamgir")
        elif (name == "Muhith" or name == "AMA Muhith" or name == "MA Muhith"):
            disambiguated_names.append("Abul Maal Abdul Muhith")
        elif (name == "Nizami" or name == "Motiur Rahman"):
            disambiguated_names.append("Motiur Rahman Nizami")
        elif (name == "Modi"):
            disambiguated_names.append("Narendra Modi")
        elif (name == "Bangabandhu" or name == "Sheikh Mujib" or name == "Sheikh Mujib" or name == "Bangabandhu Sheikh Mujibur" or name == "Sheikh Mujibur Rahman" or name == "Mujibur Rahman"):
            disambiguated_names.append("Bangabandhu Sheikh Mujibur Rahman")
        elif (name == "Tarique"):
            disambiguated_names.append("Tarique Rahman")
        elif (name == "Avijit" or name == "Roy"):
            disambiguated_names.append("Avijit Roy")
        elif (name == "Mozena"):
            disambiguated_names.append("Dan Mozena")
        elif (name == "Yunus" or name == "Mohammad Yunus"):
            disambiguated_names.append("Muhammad Yunus")
        elif (name == "Obama"):
            disambiguated_names.append("Barack Obama")
        elif (name == "Trump"):
            disambiguated_names.append("Donald Trump")
        elif (name == "Tagore" or name == "Rabindranath" or name == "Robindronath" or name == "Rabindronath" or name == "Robindranath"):
            disambiguated_names.append("Rabindranath Tagore")
        elif (name == "Karzai"):
            disambiguated_names.append("Hamid Karzai")
        elif (name == "Zardari"):
            disambiguated_names.append("Asif Ali Zardari")
        else:
            disambiguated_names.append(name)
    return (list(set(disambiguated_names)))

In [18]:
def org_disambiguation(orgs):
    disambiguated_orgs = []
    for org in orgs:
        if (org == "AL" or org == "the Awami League"):
            disambiguated_orgs.append("Awami League")
        elif (org == "EU" or org == "the European Union"):
            disambiguated_orgs.append("European Union")
        elif (org == "BCL" or org == "Chhatra League"):
            disambiguated_orgs.append("Bangladesh Chhatra League")
        elif (org == "DU"):
            disambiguated_orgs.append("Dhaka University")
        elif (org == "the High Court" or org == "The High Court"):
            disambiguated_orgs.append("High Court")
        elif (org == "the World Bank"):
            disambiguated_orgs.append("World Bank")
        elif (org == "the Supreme Court"):
            disambiguated_orgs.append("Supreme Court")
        elif (org == "UN" or org == "the United Nations"):
            disambiguated_orgs.append("United Nations")
        elif (org == "EC" or org == "the Election Commission"):
            disambiguated_orgs.append("Election Commission")
        elif (org == "the Supreme Court"):
            disambiguated_orgs.append("Supreme Court")
        elif (org == "the White House"):
            disambiguated_orgs.append("White House")
        else:
            disambiguated_orgs.append(org)
    return (list(set(disambiguated_orgs)))

In [19]:
def location_disambiguation(locations):
    disambiguated_locs = []
    for location in locations:
        if (location == "the United States" or location == "US" or location == "U.S." or location == "the United States of America" or location == "The United States" or location == "USA"):
            disambiguated_locs.append("United States")
        elif (location == "UK" or location == "the United Kingdom"):
            disambiguated_locs.append("United Kingdom")
        elif (location == "Dhaka City"):
            disambiguated_locs.append("Dhaka")
        else:
            disambiguated_locs.append(location)
    return (list(set(disambiguated_locs)))

In [20]:
df['person_entities'] = df['person_entities'].apply(name_disambiguation)
df['organization_entities'] = df['organization_entities'].apply(org_disambiguation)
df['location_entities'] = df['location_entities'].apply(location_disambiguation)

In [26]:
# df.to_pickle('Data/New Age/NewAge_ent.pkl')